In [1]:
from google.colab import drive
from PIL import Image as im
import numpy as np
import os

drive.mount("/content/gdrive")
print(os.getcwd())
os.chdir("/content/gdrive/My Drive/AML2021")
print(os.getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content
/content/gdrive/.shortcut-targets-by-id/1_nr6od538RmnZRE1gB6vQYFsrHaUgS19/AML2021


In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299

CLASSES = ['COVID', 'Normal']

x_train = np.array([])
y_train = np.array([])

for c in range(len(CLASSES)):
    _,_,files = next(os.walk(os.path.join('data/train',CLASSES[c])))
    num_files = len(files)
    print(f'{CLASSES[c]} contains {num_files} files')
    class_arr = np.array([[0,0]])
    class_arr[0][c] = 1
    print(f'class_arr for {CLASSES[c]} is {class_arr} with class index {c}')
    for idx, file in enumerate(files):
        img = im.open(os.path.join('data/train',CLASSES[c],file))
        print(f"loaded {idx} of {num_files}")
        img = img.resize((IMG_HEIGHT,IMG_WIDTH))
        img = np.array(img)
        img = np.array([img])
        if x_train.shape == (0,):
            x_train = img
            y_train = class_arr
        else:
            x_train = np.concatenate((x_train,img), axis=0)
            y_train = np.concatenate((y_train,class_arr), axis=0)
    print(f'x_train shape is {x_train.shape}, y_train shape is {y_train.shape}')
arr3d = np.stack([x_train,x_train,x_train], axis=-1)# convert N,w,h grey scale to N,w,h,3 rgb
np.save('x_train.npy', arr3d)
np.save('y_train.npy', y_train)

x_test = np.array([])
y_test = np.array([])

for c in range(len(CLASSES)):
    _,_,files = next(os.walk(os.path.join('data/test',CLASSES[c])))
    num_files = len(files)
    print(f'{CLASSES[c]} contains {num_files} files')
    class_arr = np.array([[0,0]])
    class_arr[0][c] = 1
    print(f'class_arr for {CLASSES[c]} is {class_arr} with class index {c}')
    for idx, file in enumerate(files):
        img = im.open(os.path.join('data/test',CLASSES[c],file))
        print(f"loaded {idx} of {num_files}")
        img = img.resize((IMG_HEIGHT,IMG_WIDTH))
        img = np.array(img)
        img = np.array([img])
        if x_test.shape == (0,):
            x_test = img
            y_test = class_arr
        else:
            x_test = np.concatenate((x_test,img), axis=0)
            y_test = np.concatenate((y_test,class_arr), axis=0)
    print(f'x_test shape is {x_test.shape}, y_test shape is {y_test.shape}')
arr3d = np.stack([x_test,x_test,x_test], axis=-1)# convert N,w,h grey scale to N,w,h,3 rgb
np.save('x_test.npy', arr3d)
np.save('y_test.npy', y_test)


Es wäre memory effizienter gewesen einen array der größe 16324, 299, 299, 3 zu nehmen, x_train da reinzuladen und darin x_covid und x_normal zu sortieren dann aus x_covid zu ziehen und die gezogenen a.d. freien stelln zu schreiben, aber als ich gemerkt habe das der speicher nicht ausreicht war es schneller kurz zwischen zu speichern deshalb überarbeite falls du zeit hast (also nie)

In [ ]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

print(x_train.shape, y_train.shape)

#from matplotlib import pyplot as plt
#plt.imshow(x_train[200])
#plt.show()

print("Covid, Non Covid Images:", y_train.sum(axis=0))

(11055, 299, 299, 3) (11055, 2)
Covid, Non Covid Images: [2893 8162]


In [ ]:
mask_covid = y_train[:,0]==1
mask_normal = y_train[:,1]==1
x_covid = x_train[mask_covid]
x_normal = x_train[mask_normal]
print(x_covid.shape, x_normal.shape)

(2893, 299, 299, 3) (8162, 299, 299, 3)


In [ ]:
mask_draw = np.random.randint(0, high=2893, size=8162-2893)
print(mask_draw.min())
print(mask_draw.max())
assert mask_draw.min()>=0
assert mask_draw.max()<=2892

x_covid_oversampled = np.concatenate((x_covid, x_covid[mask_draw]), axis=0) #ensure each sample is in the dataset atleast once
print(x_covid_oversampled.shape)

0
2892
(8162, 299, 299, 3)


In [ ]:
np.save('x_covid_oversampled.npy', x_covid_oversampled) #save because you cant do everything in RAM at once
np.save('x_normal.npy', x_normal)

In [ ]:
y_covid = np.zeros((8162,2), dtype="int64")
y_normal = np.zeros((8162,2), dtype="int64")
y_covid[:,0]=1
y_normal[:,1]=1
print(y_covid.shape, y_normal.shape)
print(y_covid.sum(axis=0), y_normal.sum(axis=0))

(8162, 2) (8162, 2)
[8162    0] [   0 8162]


In [ ]:
x_covid_oversampled = np.load("x_covid_oversampled.npy")
x_normal = np.load("x_normal.npy")

x = np.concatenate((x_covid_oversampled, x_normal), axis=0)
y = np.concatenate((y_covid, y_normal), axis=0)
print(x.shape, y.shape)

(16324, 299, 299, 3) (16324, 2)


In [ ]:
np.save('x_train.npy', x)
np.save('y_train.npy', y)

In [ ]:
x = np.load("x_train.npy")
y = np.load("y_train.npy")
shuffle = np.arange(0,16324)
np.random.shuffle(shuffle)
x = x[shuffle]
y = y[shuffle]

In [ ]:
print(x.shape, y.shape)
print(y[0:10])
print(y.sum(axis=0))

(16324, 299, 299, 3) (16324, 2)
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]]
[8162 8162]


In [ ]:
np.save('x_train.npy', x)
np.save('y_train.npy', y)

Preprocessing: 

In [6]:
x = np.load('x_train.npy')
print(x.shape)
np.save('x_train_part1.npy', x[0:8162])
np.save('x_train_part2.npy', x[8162:16324])


(16324, 299, 299, 3)


In [2]:
from tensorflow.keras.applications.densenet import preprocess_input
x_part = np.load("x_train_part1.npy")
x_part = preprocess_input(x_part)
np.save('x_train_part1.npy', x_part)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_part[200])
plt.show()

In [3]:
from tensorflow.keras.applications.densenet import preprocess_input
x_part = np.load("x_train_part2.npy")
x_part = preprocess_input(x_part)
np.save('x_train_part2.npy', x_part)

In [ ]:
x = np.zeros(((16324, 299, 299, 3)))
x[0:8162] = np.load('x_train_part1.npy')
x[8162:16324] = np.load('x_train_part2.npy')

print(x.shape)
from matplotlib import pyplot as plt
plt.imshow(x[200])
plt.show()
# todo speichern, ich hab mich aber für downscaling statt preprocessing anhand mean und variance der image net data entschienden
